# Importing and cleaning the customer data set

In [2]:
import pandas as pd
import numpy as np
import os

In [6]:
path = r'C:\Users\Lenovo\Desktop\Career Foundry\28-06-2024 Instacart Basket Analysis - Saswata Biswas'

In [8]:
# Import new customer data set from the 'Original Data' folder as a dataframe
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

## Q 04: Wrangling

In [10]:
#Checking the data types to see if anything can be compressed
customers.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

In [14]:
#Checking the head
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [16]:
# Changing the diplay type:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [18]:
# Checking the details:
customers.describe()

,user_id,Age,n_dependants,income
count,206209.00,206209.00,206209.00,206209.00
mean,103105.00,49.50,1.50,94632.85
std,59527.56,18.48,1.12,42473.79
min,1.00,18.00,0.00,25903.00
25%,51553.00,33.00,0.00,59874.00
50%,103105.00,49.00,1.00,93547.00
75%,154657.00,66.00,3.00,124244.00
max,206209.00,81.00,3.00,593901.00


In [20]:
# Changing the data types to compress the file:

customers['user_id']=customers['user_id'].astype('int32')
customers['Age']=customers['Age'].astype('int16')
customers['n_dependants']=customers['n_dependants'].astype('int8')
customers['income']=customers['income'].astype('int32')

In [22]:
#Checking whether it worked:
customers.dtypes

user_id          int32
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int16
date_joined     object
n_dependants      int8
fam_status      object
income           int32
dtype: object

In [24]:
# Changing names to match the other ors_prods file:

customers.rename(columns = {'First Name' : 'first_name'}, inplace = True)
customers.rename(columns = {'Surnam' : 'surname'}, inplace = True)
customers.rename(columns = {'Gender' : 'gender'}, inplace = True)
customers.rename(columns = {'Age' : 'age'}, inplace = True)
customers.rename(columns = {'STATE' : 'state'}, inplace = True)
customers.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

## Q 05: Check for Dups or missing values or mixed data types

In [26]:
# Checking for mixed data-types in each column
for col in customers.columns.tolist():
  weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

first_name


In [28]:
# Changing the data type for first_name:

customers['first_name']=customers['first_name'].astype('str')

In [30]:
# Checking the file after all the changes:

customers.dtypes

user_id            int32
first_name        object
surname           object
gender            object
state             object
age                int16
date_joined       object
n_dependants        int8
marital_status    object
income             int32
dtype: object

In [37]:
# Check for missing data
customers.isnull().sum()

user_id           0
first_name        0
surname           0
gender            0
state             0
age               0
date_joined       0
n_dependants      0
marital_status    0
income            0
dtype: int64

In [39]:
# Check for full-duplicates (first create a new df)
customers_dups = customers[customers.duplicated()]

In [41]:
#Check the new DF
customers_dups

,user_id,first_name,surname,gender,state,age,date_joined,n_dependants,marital_status,income


#### Good news: No duplicates are found

## We will not be dropping any column as they might reveal further insight, for example, a person with more dependants might tend to buy more produce

## Q 06: Combine your customer data with the rest of your prepared Instacart data. (Hint: Make sure the key columns are the same data type!)

In [46]:
# Step 1 Import the latest ords_prods_file


In [48]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_aggregated.pkl'))

In [50]:
# Checking data types and looking for the key column:
ords_prods_merge.dtypes

order_id                     int32
user_id                      int32
order_number                  int8
orders_days_of_week           int8
order_hour_of_day             int8
days_since_prior_order     float16
product_id                   int32
add_to_cart_order            int16
reordered                     int8
product_name                object
aisle_id                     int16
department_id                 int8
prices                     float64
_merge                    category
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                     int8
loyalty_flag                object
average_spend              float64
spender_flag                object
customer_frequency         float16
frequency_flag              object
dtype: object

### We can use the user_id as the key column as they are available in both data sets

In [53]:
# Drop the existing '_merge' column from previous exercises
ords_prods_merge = ords_prods_merge.drop(columns=['_merge'])

In [55]:
# Merge the two dataframes using the column 'user_id' as the shared column with the default join (inner)

cust_ord_prod = ords_prods_merge.merge(customers, on = 'user_id', indicator = True)

In [57]:
# Checking to see the newly produced DF:
cust_ord_prod.shape

(32404859, 33)

In [59]:
cust_ord_prod.dtypes

order_id                     int32
user_id                      int32
order_number                  int8
orders_days_of_week           int8
order_hour_of_day             int8
days_since_prior_order     float16
product_id                   int32
add_to_cart_order            int16
reordered                     int8
product_name                object
aisle_id                     int16
department_id                 int8
prices                     float64
price_range_loc             object
busiest_day                 object
busiest_days                object
busiest_period_of_day       object
max_order                     int8
loyalty_flag                object
average_spend              float64
spender_flag                object
customer_frequency         float16
frequency_flag              object
first_name                  object
surname                     object
gender                      object
state                       object
age                          int16
date_joined         

In [61]:
# Check inner join value counts and whether there are any outliers
cust_ord_prod['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

### The count of both and the count of rows match

## Q 08: Export this new dataframe as a pickle file so you can continue to use it in the second part of this task.

In [65]:
cust_ord_prod.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers_merged.pkl'))

#### Checked the Prepared Data folder and the file has been exported succesfully